<a href="https://colab.research.google.com/github/elephanti/NLPProject2024/blob/main/LAMBADA_Data_Augmentation_Mistral_7B_v0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAMBADA - Fine tuning Mistral 7B for data augmentation

## Installations & Imports

In [1]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U accelerate
!pip install -U peft
!pip install -U datasets
!pip install -U evaluate
!pip install -U trl
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import Dataset, load_dataset
from trl import SFTTrainer
import pandas as pd
from sklearn.utils import shuffle
from huggingface_hub import notebook_login

## Login

In [13]:
notebook_login()

In [4]:
wandb.login(key = "7f4d8d8963c96f20cb342cdf530b65db738b4e18")
run = wandb.init(
    project='LAMBADA Data Augmentation - Mistral 7B v0.3 Quantified',
    job_type="training",
    anonymous="allow"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: elephanti12345 (elephanti12345-runi). Use `wandb login --relogin` to force relogin


## Dataset loading & preprocessing

In [5]:
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/ATIS/sampled_subsets/ver1/atis_10_subset.csv -P /content/datasets/ATIS/VER1

--2024-07-18 09:54:18--  https://github.com/elephanti/NLPProject2024/raw/main/datasets/ATIS/sampled_subsets/ver1/atis_10_subset.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/elephanti/NLPProject2024/main/datasets/ATIS/sampled_subsets/ver1/atis_10_subset.csv [following]
--2024-07-18 09:54:18--  https://raw.githubusercontent.com/elephanti/NLPProject2024/main/datasets/ATIS/sampled_subsets/ver1/atis_10_subset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9956 (9.7K) [text/plain]
Saving to: ‘/content/datasets/ATIS/VER1/atis_10_subset.csv’

atis_10_subset.csv  100%[===================>]   

In [6]:
BOS_TOKEN = "<s>"
EOS_TOKEN = "</s>"
SEP_TOKEN = "[SEP]"

In [7]:
def load_and_preprocess_dataset(filename:str) -> Dataset:
    df = pd.read_csv(filename)
    df['text'] = df.apply(lambda row: f"{BOS_TOKEN}{row['label']} {SEP_TOKEN} {row['text']}{EOS_TOKEN}", axis=1)
    df = shuffle(df)
    dataset = Dataset.from_pandas(df)
    return dataset

In [8]:
train_dataset = load_and_preprocess_dataset('datasets/ATIS/VER1/atis_10_subset.csv')

In [9]:
train_dataset['text'][:5]


['<s>abbreviation [SEP] what is fare code m</s>',
 '<s>abbreviation [SEP] what is fare code qw</s>',
 '<s>abbreviation [SEP] what does lax stand for</s>',
 '<s>abbreviation [SEP] what does fare code y mean</s>',
 '<s>abbreviation [SEP] show me what fare codes symbols stand for</s>']

## Finetune Mistral 7B v0.3

In [10]:
BASE_MODEL = "mistralai/Mistral-7B-v0.3"
NEW_MODEL = "Mistral-7B-v0.3-atis-10-lambada-aug"

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=True
)

model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True, use_auth_token=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.unk_token
tokenizer.add_eos_token = False
tokenizer.add_bos_token = False
tokenizer.model_max_length = 1024

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [16]:
config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)

model = get_peft_model(model, config)

In [17]:
model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.resize_token_embeddings(len(tokenizer))

Embedding(32768, 4096)

In [18]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2.5e-5,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

In [19]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    dataset_text_field='text',
    max_seq_length=tokenizer.model_max_length,
    tokenizer=tokenizer,
    packing=False
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/157 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,4.029100
2,4.321200
3,4.680800
4,4.356900
5,4.751400
6,4.935000
7,4.569300
8,6.166200
9,5.109000
10,3.672200


TrainOutput(global_step=160, training_loss=2.344516611844301, metrics={'train_runtime': 434.9501, 'train_samples_per_second': 1.444, 'train_steps_per_second': 0.368, 'total_flos': 582827791810560.0, 'train_loss': 2.344516611844301, 'epoch': 4.0})

In [20]:
trainer.model.save_pretrained(NEW_MODEL)
wandb.finish()
model.config.use_cache = True
model.eval()

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,▂▄▃▅▅▇▄▂▂▁▂▂▁▂▅▁▅▂▂▂▁▂▂▂▄▂▃▁▄▃▂▂▃▂▆▃▂▃▃█
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,▇█▆▆▇▄▅▄▅▄▃▃▂▃▃▂▄▃▃▃▂▂▂▂▃▂▃▂▂▂▁▁▂▁▂▁▁▁▁▁
total_flos,582827791810560.0
train/epoch,4.0
train/global_step,160
train/grad_norm,7.11611
train/learning_rate,3e-05
train/loss,1.5687


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): 

In [21]:
tokenizer.save_pretrained(NEW_MODEL)

('Mistral-7B-v0.3-atis-10-lambada-aug/tokenizer_config.json',
 'Mistral-7B-v0.3-atis-10-lambada-aug/special_tokens_map.json',
 'Mistral-7B-v0.3-atis-10-lambada-aug/tokenizer.model',
 'Mistral-7B-v0.3-atis-10-lambada-aug/added_tokens.json',
 'Mistral-7B-v0.3-atis-10-lambada-aug/tokenizer.json')

In [22]:
model.push_to_hub(NEW_MODEL, use_auth_token=True)
tokenizer.push_to_hub(NEW_MODEL, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ALivshits/Mistral-7B-v0.3-atis-10-lambada-aug/commit/ed799efbe9c72fe6160918992add62e76c4b9f50', commit_message='Upload tokenizer', commit_description='', oid='ed799efbe9c72fe6160918992add62e76c4b9f50', pr_url=None, pr_revision=None, pr_num=None)

## Generate synthetic data

In [51]:
def generate_samples(intents, model, tokenizer, num_samples=10):
    inputs = [f'{BOS_TOKEN}{intent} {SEP_TOKEN}' for intent in intents]
    tokenized_prompts = tokenizer(inputs, padding=True, return_tensors='pt')
    model_op = model.generate(input_ids=tokenized_prompts['input_ids'].to('cuda'),
                              attention_mask=tokenized_prompts['attention_mask'].to('cuda'),
                              min_length=10,
                              max_length=100,
                              temperature=1,
                              top_k=30,
                              top_p=0.90,
                              repetition_penalty=1.5,
                              do_sample=True,
                              num_return_sequences=10,
                              use_cache=True)
    generated_text = tokenizer.batch_decode(model_op, skip_special_tokens=True)
    samples = [item.split(SEP_TOKEN) for item in generated_text]
    samples = [(label.strip(), text.strip()) for (label, text) in samples]
    return pd.DataFrame(samples, columns=['label', 'text'])

In [44]:
intents = train_dataset.to_pandas()['label'].unique()
intents

array(['abbreviation', 'aircraft', 'airfare', 'airline', 'airport',
       'capacity', 'cheapest', 'city', 'day_name', 'distance', 'flight',
       'flight_no', 'flight_time', 'ground_fare', 'ground_service',
       'meal', 'quantity', 'restriction'], dtype=object)

In [52]:
intents = train_dataset.to_pandas()['label'].unique()
generated_samples = generate_samples(intents, model, tokenizer)
generated_samples

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


,label,text
0,abbreviation,what does ua stand for
1,abbreviation,what does fare code y mean
2,abbreviation,what does ua stand for
3,abbreviation,what does fare code y mean
4,abbreviation,what does yn stand for
...,...,...
175,restriction,apa restriction on travel from canada to the us
176,restriction,can i go from baltimore to pittsburgh on ameri...
177,restriction,flights from san francisco to boston before 8 am
178,restriction,are there any restrictions from oakland to boston


In [49]:
# Extract the 'text' column from both DataFrames
train_texts = train_dataset.to_pandas()['text']
generated_texts = generated_samples['text']

# Find duplicates
duplicates = set(train_texts).intersection(generated_texts)

if duplicates:
  print("Duplicates found:")
  for duplicate in duplicates:
    print(duplicate)
else:
  print("No duplicates found between generated samples and training dataset.")


No duplicates found between generated samples and training dataset.
